In [8]:
import numpy as np

In [5]:
import tensorflow as tf
import json

class ChatbotModel():
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.max_sequence_len = None
        self.total_words = None

    def train(self, data):
        corpus = data.lower().split("\n")

        self.tokenizer = tf.keras.preprocessing.text.Tokenizer()
        self.tokenizer.fit_on_texts(corpus)
        self.total_words = len(self.tokenizer.word_index) + 1

        input_sequences = []
        for line in corpus:
            token_list = self.tokenizer.texts_to_sequences([line])[0]
            for i in range(1, len(token_list)):
                n_gram_sequence = token_list[:i + 1]
                input_sequences.append(n_gram_sequence)

        self.max_sequence_len = max(len(x) for x in input_sequences)
        input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=self.max_sequence_len, padding='pre')
        xs = input_sequences[:, :-1]
        labels = input_sequences[:, -1]
        ys = tf.keras.utils.to_categorical(labels, num_classes=self.total_words)

        self.model = tf.keras.models.Sequential()
        self.model.add(tf.keras.layers.Embedding(self.total_words, 240, input_length=self.max_sequence_len - 1))
        self.model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)))
        self.model.add(tf.keras.layers.Dense(self.total_words, activation='softmax'))
        adam = tf.keras.optimizers.Adam(learning_rate=0.01)
        self.model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
        history = self.model.fit(xs, ys, epochs=30, verbose=1)

        # Save the model as a SavedModel
        tf.saved_model.save(self.model, "chatbot_model")

        # Convert the model to TensorFlow Lite
        converter = tf.lite.TFLiteConverter.from_saved_model("chatbot_model")
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
        converter.experimental_lower_tensor_list_ops = False
        tflite_model = converter.convert()


        # Save the TensorFlow Lite model to a file
        with open("chatbot_model.tflite", "wb") as f:
            f.write(tflite_model)

        # Save the tokenizer word index
        with open("tokenizer_word_index.json", "w") as f:
            json.dump(self.tokenizer.word_index, f)

    def predict(self, user_input):
        # Load the tokenizer word index
        with open("tokenizer_word_index.json", "r") as f:
            word_index = json.load(f)

        tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer.word_index = word_index

        # Load the TensorFlow Lite model
        interpreter = tf.lite.Interpreter(model_path="chatbot_model.tflite")
        interpreter.allocate_tensors()

        # Get input and output details
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Perform inference
        seed_text = user_input
        next_word = " "
        answer = ""

        while next_word != "ouavjra":
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=input_details[0]['shape'][1], padding='pre')

            # Set input tensor
            input_data = token_list.astype(np.float32)
            interpreter.set_tensor(input_details[0]['index'], input_data)

            # Run inference
            interpreter.invoke()

            # Get the output tensor
            output_data = interpreter.get_tensor(output_details[0]['index'])

            predicted = np.argmax(output_data, axis=1)
            output_word = ""
            for word, index in tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " " + output_word
            next_word = output_word

        try:
            answer = seed_text.split("arjvauo")[1][:-7]
        except:
            answer = "Pardon! I didn't find your answer."

        return answer


In [6]:
if __name__ == '__main__':
    chatbot = ChatbotModel()
    data = '''Hello ARJVAUO hello, i am A I chat bot. How may i help you . OUAVJRA
What is IIC ? ARJVAUO Ministry of Human Resource Development, Govt. of India has established ‘MHRD’s Innovation Cell (MIC)’ to systematically foster the culture of Innovation amongst all Higher Education Institutions (HEIs). The primary mandate of MIC is to encourage, inspire and nurture young students by supporting them to work with new ideas and transform them into prototypes while they are informative years OUAVJRA
Good morning ARJVAUO Good morning to you too! I'm here to assist you with any questions or concerns you may have. OUAVJRA
Good Afternoon ARJVAUO Good afternoon to you too! I'm here to assist you with any questions or concerns you may have. OUAVJRA
Good evening ARJVAUO Good evening to you too! I'm here to assist you with any questions or concerns you may have. OUAVJRA
What's going on? ARJVAUO Not much is going on with me since I'm an AI language model. But if you have any questions or concerns, I'm here to help. OUAVJRA
What's new? ARJVAUO Nothing new on my end, but I'm ready to help you with whatever you need. OUAVJRA
What have you been up to lately? ARJVAUO As an AI language model, I don't have personal experiences or activities. However, I'm always here to help you with any questions or concerns you may have. OUAVJRA
How are you doing today? ARJVAUO I'm an AI language model, so I don't have emotions like humans do. But thank you for asking! How can I assist you today? OUAVJRA
What is your name? ARJVAUO My name is AI Chatbot OUAVJRA
What is the capital of India? ARJVAUO New Delhi OUAVJRA
What is the weather like today? ARJVAUO  I'm sorry, but I don't have access to real-time weather data. You can check your local weather forecast or a weather website for more information. OUAVJRA
What is the meaning of life? ARJVAUO The meaning of life is a philosophical and subjective question that has been debated for centuries. Many people find their own meaning in life through their personal experiences and beliefs. OUAVJRA
How do you work? ARJVAUO  I work by using deep learning algorithms to process and understand natural language inputs, and then generate appropriate responses based on my programming and training. OUAVJRA
Can you tell me a joke? ARJVAUO Sure, here's one: Why did the tomato turn red? Because it saw the salad dressing! OUAVJRA
What is your purpose? ARJVAUO My purpose is to answer questions and provide information to the best of my abilities. OUAVJRA
okay bye ARJVAUO Bye, Have a nice day OUAVJRA
how are you? ARJVAUO hey I am fine, what about you OUAVJRA
fine ARJVAUO Great, how can i help you ? OUAVJRA
Hey ARJVAUO Hi, How are you? OUAVJRA
I am fine ARJVAUO Great, How can I help you ? OUAVJRA
I am good  ARJVAUO Great, How can I help you ? OUAVJRA
Thank you ARJVAUO welcome OUAVJRA
how are you? ARJVAUO hey I am fine, what about you OUAVJRA
fine ARJVAUO Great, how can i help you ? OUAVJRA
What is MYIIEST? ARJVAUO An Central Engineering Institution OUAVJRA
What is NIRF rank? ARJVAUO 41 OUAVJRA
How many hostels are there? ARJVAUO 71 OUAVJRA
When IIEST has been stablished? ARJVAUO 1856 OUAVJRA
Where is CS department? ARJVAUO main building OUAVJRA
Who is the HOD of cs? ARJVAUO Nirnay Sir OUAVJRA
How many students are enrolled? ARJVAUO 5000 OUAVJRA
What is highest package of 2021? ARJVAUO 50LPA OUAVJRA
what is average package of cs? ARJVAUO 19LPA OUAVJRA
show me, today's class routein ARJVAUO follow the link OUAVJRA
what is jee mains cutoff of 2021 of cs? ARJVAUO 99%tile OUAVJRA
What are the Academic Courses IIEST offers for UG? ARJVAUO	B.Tech.(4years),B.Arch.(5years) and Dual Degree(B.Tech.-M.Tech) for 5 years OUAVJRA
Is IIEST a NIT? ARJVAUO	IIEST is an NIT and its controlled by the Council by NITSER. OUAVJRA
IIEST is private or Govt College? ARJVAUO IIEST in under the Central Government. OUAVJRA
Does IIEST allow other external Scholarships? ARJVAUO YES, NSP,Reliance,Swami Vivekananda Scholarship and etc.Contact  Abhijit Bannerjee (SnT-5th floor) or Dean Academic Ashok Adak Sir (Main-1st floor) OUAVJRA
Does IIEST provide Tution-Fee-Wavier oppertunities? ARJVAUO YES,Full-Tution-Fee-Wavier & Partial-Tution-Fee-Wavier OUAVJRA
Who are Eligible for Full and Partial Tution Fee Wavier? ARJVAUO Full-TFW:Annual Income<1,00,000 & Partial-TFW:Annual Income:1,00,000 to 5,00,000 OUAVJRA
When should i get the Refund of JOSAA-Admission-fees? ARJVAUO Latest by 3rd Semester(must make college bank account) else contact Abhijit Bannerjee(SnT-5th floor) OUAVJRA
Does IIEST allow studying through Bank Loan and how? ARJVAUO YES,Mail Dean-of-Students-Welfare for Bonafide-Certificate complete rest process at your bank and forward College Bank details during Semester payment. OUAVJRA
How many Banks are there at IIEST Shibpur? ARJVAUO UCO(SnT building),UBI and SBI(near Oval Ground) OUAVJRA
Hi ARJVAUO Hello, Welcome to IIEST any Questions ? OUAVJRA
What are the Clubs at IIEST Shibpur? ARJVAUO Robodarsha,Les Thespians,Catharsis,Euphony,LitSoc,Quizmaniac BEings,SCAGE,Society of Games and Sports,Code IIEST,DebSoc,Reflexo Beta,Entrepreneurship Development Cell,Camerabuff,Aeromodelling Club OUAVJRA
Whats Special about IIEST? ARJVAUO The college has a rich culture of heritage with foundations of historical importance and a green lush campus. OUAVJRA
How many Trees are there at IIEST? ARJVAUO 2610 Trees OUAVJRA
What are the Fests at IIEST Shibpur? ARJVAUO REBECA,GYANDHARA,OIKOTAA,ANURANAN,TEDX-IIESTSHIBPUR,UDBHAVANI,INITIUM,METALLUM,SPHURAN,SCINTILLATIONS,IMPETUS,VAIDYUTIK,ABHIYANTRIX,SATRANG,KRITI. OUAVJRA
Does IIEST provide Scholarships ? ARJVAUO GAABESU Scholarship Programme OUAVJRA
bad answer ARJVAUO I am sorry but I prefer not to continue this conversation. I am still learning so I appreciate your understanding and patience.🙏 OUAVJRA
how many departments are ? ARJVAUO 20departments OUAVJRA'''
    chatbot.train(data)

    # print("Hi")
    # user_input = input()
    # chatbot.predict(user_input)

Epoch 1/30
29/29 [==============================] - 9s 193ms/step - loss: 5.8456 - accuracy: 0.0562
Epoch 2/30
29/29 [==============================] - 6s 193ms/step - loss: 5.0487 - accuracy: 0.0908
Epoch 3/30
29/29 [==============================] - 6s 194ms/step - loss: 4.0614 - accuracy: 0.1849
Epoch 4/30
29/29 [==============================] - 6s 191ms/step - loss: 3.0092 - accuracy: 0.3384
Epoch 5/30
29/29 [==============================] - 6s 195ms/step - loss: 1.9923 - accuracy: 0.5222
Epoch 6/30
29/29 [==============================] - 6s 189ms/step - loss: 1.1860 - accuracy: 0.7146
Epoch 7/30
29/29 [==============================] - 6s 192ms/step - loss: 0.6710 - accuracy: 0.8346
Epoch 8/30
29/29 [==============================] - 6s 190ms/step - loss: 0.4061 - accuracy: 0.8876
Epoch 9/30
29/29 [==============================] - 6s 190ms/step - loss: 0.2993 - accuracy: 0.9178
Epoch 10/30
29/29 [==============================] - 6s 203ms/step - loss: 0.2429 - accuracy: 0.9222

INFO:tensorflow:Assets written to: chatbot_model\assets


INFO:tensorflow:Assets written to: chatbot_model\assets


In [9]:
chatbot.predict("Hello")

' hello i am a i chat bot how may i help you '